# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/medgemma_kaggle_competition/blob/main/medgemma_impact_starter.ipynb)

# Winning the MedGemma Impact Challenge with FiftyOne

**The difference between a demo and a winning submission is understanding where your model breaks—and why.**

This notebook shows you how to use FiftyOne as your workbench for the MedGemma Impact Challenge. 
We'll go beyond running inference and printing metrics. You'll learn to:

1. **Explore your data** before modeling
2. **Visualize embeddings** to diagnose learnability  
3. **Run MedGemma inference** and store predictions alongside ground truth
4. **Analyze failures** systematically—not just count them
5. **Export curated subsets** for fine-tuning

We'll use the [SLAKE dataset](https://huggingface.co/datasets/Voxel51/SLAKE), a medical VQA benchmark 
with images from multiple modalities (CT, MRI, X-ray), rich annotations including bounding boxes and 
segmentation masks, and questions spanning anatomy, abnormalities, and more.

---

## Setup & Installation

In [ ]:
# !pip install -U fiftyone huggingface_hub accelerate sentencepiece protobuf torch torchvision

In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from fiftyone import ViewField as F

### Authenticate with Hugging Face

Both MedGemma and MedSigLIP are gated models. You'll need to:
1. Request access on [MedGemma](https://huggingface.co/google/medgemma-1.5-4b-it) and [MedSigLIP](https://huggingface.co/google/medsiglip-448)
2. Set your HF token

In [ ]:
import os
# os.environ["HF_TOKEN"] = "your_token_here"

# Or login via CLI: huggingface-cli login

---
## 1. Load the SLAKE Dataset

The SLAKE dataset is already in FiftyOne format on Hugging Face. One line to load it.

In [ ]:
from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "Voxel51/SLAKE",
    name="SLAKE",
    overwrite=True,
)

print(f"Loaded {len(dataset)} samples")
print(dataset)

Let's look at the schema to understand what we're working with:

In [ ]:
dataset.print_field_schema()

Each sample has:
- **question**: The VQA question
- **answer**: Ground truth answer
- **question_type**: "open" or "closed" (free-form vs multiple choice)
- **content_type**: What the question is about (e.g., "Organ", "Abnormality", "Position")
- **modality**: Imaging modality (CT, MRI, X-Ray)
- **body_part**: Anatomical region
- **mask**: Segmentation mask (where available)
- **detections**: Bounding box annotations with labels (where available)

---
## 2. Explore Your Data (Before You Model)

Don't rush to inference. Understanding your data distribution is how you catch problems early.

### What modalities do we have?

In [ ]:
print("Modalities:")
for modality, count in dataset.count_values("modality").items():
    print(f"  {modality}: {count}")

### What types of questions?

In [ ]:
print("\nQuestion Types:")
for qtype, count in dataset.count_values("question_type").items():
    print(f"  {qtype}: {count}")

print("\nContent Types:")
for ctype, count in sorted(dataset.count_values("content_type").items(), key=lambda x: -x[1]):
    print(f"  {ctype}: {count}")

### What body parts are covered?

In [ ]:
print("\nBody Parts:")
for part, count in sorted(dataset.count_values("body_part").items(), key=lambda x: -x[1]):
    print(f"  {part}: {count}")

### Launch the App to explore visually

This is where FiftyOne shines. Filter, sort, and browse your data interactively.

In [ ]:
session = fo.launch_app(dataset)

**Try these in the App:**
- Filter to `modality == "CT"` 
- Filter to `content_type == "Abnormality"`
- Look at samples with bounding boxes vs without

You'll start to notice patterns: certain body parts have more questions, 
certain modalities are over/under-represented, etc.

### Create useful views

Save views for subsets you'll come back to:

In [ ]:
# Closed questions only (easier to evaluate)
closed_questions = dataset.match(F("question_type") == "closed")
print(f"Closed questions: {len(closed_questions)}")

# Questions with localization annotations
has_detections = dataset.match(F("detections") != None)
print(f"Samples with detections: {len(has_detections)}")

# CT images only
ct_only = dataset.match(F("modality") == "CT")
print(f"CT images: {len(ct_only)}")

---
## 3. Compute Embeddings with MedSigLIP

Before running VQA inference, let's see if the embedding space even separates our classes.
If MedSigLIP embeddings don't cluster by modality or body part, that's diagnostic information.

### Register and load MedSigLIP

In [ ]:
# Register the model source (one time)
foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/medsiglip",
    overwrite=True
)

# Download the model (one time)
foz.download_zoo_model(
    "https://github.com/harpreetsahota204/medsiglip",
    model_name="google/medsiglip-448",
)

In [ ]:
# Load the model
medsiglip = foz.load_zoo_model("google/medsiglip-448")

### Compute embeddings

In [ ]:
dataset.compute_embeddings(
    model=medsiglip,
    embeddings_field="medsiglip_embeddings",
)

print("Embeddings computed!")

### Visualize in 2D

Project to 2D with UMAP and see if natural clusters emerge:

In [ ]:
results = fob.compute_visualization(
    dataset,
    embeddings="medsiglip_embeddings",
    method="umap",
    brain_key="medsiglip_viz",
    num_dims=2,
)

In [ ]:
# Relaunch app to see embeddings panel
session = fo.launch_app(dataset)

**In the App:**
- Open the Embeddings panel
- Color by `modality` — do CT, MRI, X-ray form distinct clusters?
- Color by `body_part` — do anatomical regions separate?
- Color by `content_type` — do question types cluster?

**What you're looking for:**
- Clear separation = model has a chance
- Everything mixed together = fundamental representation problem

### Build a similarity index for later

This lets us find similar images, which is useful for error analysis.

In [ ]:
sim_index = fob.compute_similarity(
    dataset,
    model="google/medsiglip-448",
    brain_key="medsiglip_similarity",
    embeddings="medsiglip_embeddings"
)

---
## 4. Run MedGemma Inference

Now let's run MedGemma 1.5 on the VQA task and store predictions.

### Register and load MedGemma

In [ ]:
foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/medgemma_1_5",
    overwrite=True
)

foz.download_zoo_model(
    "https://github.com/harpreetsahota204/medgemma_1_5",
    model_name="google/medgemma-1.5-4b-it",
)

In [ ]:
medgemma = foz.load_zoo_model("google/medgemma-1.5-4b-it")

### Configure for VQA

In [ ]:
# Set operation mode
medgemma.operation = "vqa"

### Run inference using per-sample prompts

Each sample has its own question stored in the `question` field. 
We use `prompt_field` to read prompts from the dataset.

In [ ]:
# Run on full dataset (or a subset for testing)
# For faster iteration, start with a subset:
# subset = dataset.take(100)

dataset.apply_model(
    medgemma,
    label_field="medgemma_answer",
    prompt_field="question",  # Each sample uses its own question
    batch_size=32,
    num_workers=4,
)

print("Inference complete!")

### Inspect predictions

In [ ]:
# Look at a few samples
for sample in dataset.take(5):
    print(f"Q: {sample.question}")
    print(f"GT: {sample.answer}")
    print(f"Pred: {sample.medgemma_answer.label if sample.medgemma_answer else 'None'}")
    print("-" * 50)

---
## 5. Evaluate Performance

Let's compute accuracy—but more importantly, let's slice it to find patterns.

### Add correctness field

In [ ]:
# For closed questions, we can do exact match
# For open questions, you'd want semantic similarity

def normalize_answer(ans):
    """Basic normalization for answer comparison"""
    if ans is None:
        return ""
    return str(ans).lower().strip()

# Add is_correct field
for sample in dataset.iter_samples(autosave=True):
    gt = normalize_answer(sample.answer)
    pred = normalize_answer(
        sample.medgemma_answer.label if sample.medgemma_answer else None
    )
    sample["is_correct"] = (gt == pred)

### Overall accuracy

In [ ]:
correct = dataset.match(F("is_correct") == True)
total = len(dataset)
accuracy = len(correct) / total

print(f"Overall Accuracy: {accuracy:.2%} ({len(correct)}/{total})")

### Accuracy by question type

Closed questions should be easier than open-ended ones.

In [ ]:
for qtype in dataset.distinct("question_type"):
    view = dataset.match(F("question_type") == qtype)
    correct_view = view.match(F("is_correct") == True)
    acc = len(correct_view) / len(view) if len(view) > 0 else 0
    print(f"{qtype}: {acc:.2%} ({len(correct_view)}/{len(view)})")

### Accuracy by modality

Does MedGemma perform differently on CT vs MRI vs X-ray?

In [ ]:
print("\nAccuracy by Modality:")
for modality in dataset.distinct("modality"):
    view = dataset.match(F("modality") == modality)
    correct_view = view.match(F("is_correct") == True)
    acc = len(correct_view) / len(view) if len(view) > 0 else 0
    print(f"  {modality}: {acc:.2%} ({len(correct_view)}/{len(view)})")

### Accuracy by body part

In [ ]:
print("\nAccuracy by Body Part:")
results = []
for part in dataset.distinct("body_part"):
    view = dataset.match(F("body_part") == part)
    correct_view = view.match(F("is_correct") == True)
    acc = len(correct_view) / len(view) if len(view) > 0 else 0
    results.append((part, acc, len(view)))

# Sort by accuracy
for part, acc, n in sorted(results, key=lambda x: x[1]):
    print(f"  {part}: {acc:.2%} (n={n})")

### Accuracy by content type

In [ ]:
print("\nAccuracy by Content Type:")
results = []
for ctype in dataset.distinct("content_type"):
    view = dataset.match(F("content_type") == ctype)
    correct_view = view.match(F("is_correct") == True)
    acc = len(correct_view) / len(view) if len(view) > 0 else 0
    results.append((ctype, acc, len(view)))

for ctype, acc, n in sorted(results, key=lambda x: x[1]):
    print(f"  {ctype}: {acc:.2%} (n={n})")

**This is where it gets interesting.** 

You might find things like:
- "MedGemma struggles on Position questions for Abdomen CT"  
- "Abnormality detection is worse on MRI than X-ray"

These are *actionable insights*, not just numbers.

---
## 6. Error Analysis

Now let's actually look at the failures. This is where you learn something.

### Create error view

In [ ]:
errors = dataset.match(F("is_correct") == False)
print(f"Total errors: {len(errors)}")

### Launch App focused on errors

In [ ]:
session = fo.launch_app(errors)

**In the App, explore:**
- What do the failed samples look like?
- Are there patterns? (e.g., low contrast images, unusual anatomy)
- Does the model consistently fail on certain question phrasings?

### Find the worst slices

Which combinations of (modality, body_part, content_type) have the worst accuracy?

In [ ]:
from collections import defaultdict

slice_stats = defaultdict(lambda: {"correct": 0, "total": 0})

for sample in dataset.iter_samples():
    key = (sample.modality, sample.body_part, sample.content_type)
    slice_stats[key]["total"] += 1
    if sample.is_correct:
        slice_stats[key]["correct"] += 1

# Find worst performing slices (min 10 samples)
worst_slices = []
for key, stats in slice_stats.items():
    if stats["total"] >= 10:
        acc = stats["correct"] / stats["total"]
        worst_slices.append((key, acc, stats["total"]))

print("Worst performing slices (n >= 10):")
for (modality, body_part, content_type), acc, n in sorted(worst_slices, key=lambda x: x[1])[:10]:
    print(f"  {modality} / {body_part} / {content_type}: {acc:.2%} (n={n})")

### Tag error patterns

As you review errors, tag them with patterns you notice:

In [ ]:
# Example: Tag errors where model gave opposite answer for yes/no questions
yes_no_errors = errors.match(
    (F("answer").is_in(["yes", "no", "Yes", "No"])) & 
    (F("medgemma_answer.label").is_in(["yes", "no", "Yes", "No"]))
)

for sample in yes_no_errors.iter_samples(autosave=True):
    gt = sample.answer.lower()
    pred = sample.medgemma_answer.label.lower() if sample.medgemma_answer else ""
    if (gt == "yes" and pred == "no") or (gt == "no" and pred == "yes"):
        sample.tags.append("opposite_answer")

print(f"Tagged {len(dataset.match(F('tags').contains('opposite_answer')))} samples with 'opposite_answer'")

### Find similar images to errors

Use the similarity index to find images similar to failures. 
Are the similar images also failures? That suggests a systematic issue.

In [ ]:
# Get a specific error sample
error_sample = errors.first()

# Find similar images
similar_view = sim_index.sort_by_similarity(error_sample.id, k=10)

# Check how many of the similar images are also errors
similar_errors = similar_view.match(F("is_correct") == False)
print(f"Error sample: {error_sample.question}")
print(f"Similar images that are also errors: {len(similar_errors)}/10")

In [ ]:
# View the similar images
session = fo.launch_app(similar_view)

---
## 7. Fine-Tuning MedGemma for Localization

You've explored the data, identified failure patterns, and have hypotheses about what to fix.
Now let's fine-tune MedGemma to output bounding box coordinates for localization tasks.

We'll follow these steps:
1. Define a `GetItem` subclass to extract and transform data from FiftyOne
2. Create train/val splits and flatten detections using `to_patches()`
3. Set up QLoRA fine-tuning with the TRL library's `SFTTrainer`

### Install fine-tuning dependencies

In [ ]:
# !pip install --upgrade --quiet bitsandbytes peft trl

### Step 1: Define the GetItem subclass

FiftyOne's [`GetItem`](https://docs.voxel51.com/api/fiftyone.utils.torch.html#fiftyone.utils.torch.GetItem) 
class is the bridge between FiftyOne and PyTorch. It tells FiftyOne:
1. What fields to extract from each sample (via `required_keys`)
2. How to transform them into your desired format (via `__call__`)

The `field_mapping` parameter is important when working with patches. In a patches view,
the detection data lives in the original field name (e.g., "detections"), but we want 
to access it with a generic name in our code.

In [ ]:
from typing import Any
from PIL import Image
from fiftyone.utils.torch import GetItem

# System prompt for localization task
LOCALIZATION_SYSTEM_PROMPT = """Instructions:
The following user query will require outputting bounding boxes. The format of bounding boxes coordinates is [y0, x0, y1, x1] where (y0, x0) must be top-left corner and (y1, x1) the bottom-right corner. This implies that x0 < x1 and y0 < y1. Always normalize the x and y coordinates the range [0, 1000], meaning that a bounding box starting at 15% of the image width would be associated with an x coordinate of 150. You MUST output a single parseable json list of objects enclosed into ```json...``` brackets, for instance ```json[{"box_2d": [800, 3, 840, 471], "label": "car"}, {"box_2d": [400, 22, 600, 73], "label": "dog"}]``` is a valid output. Now answer to the user query.

Remember "left" refers to the patient's left side where the heart is and sometimes underneath an L in the upper right corner of the image."""


class LocalizationGetItem(GetItem):
    """
    Extracts and transforms detection data for MedGemma localization fine-tuning.
    
    Each patch sample (after to_patches()) contains:
    - filepath: path to the full image
    - detection: the Detection object (bbox, label, etc.)
    - metadata: image dimensions
    
    We transform this into MedGemma's expected message format with:
    - System prompt explaining the bbox output format
    - User message with the localization query
    - Assistant message with the target bbox in JSON format
    """
    
    def __init__(self, field_mapping=None):
        super().__init__(field_mapping=field_mapping)
    
    @property
    def required_keys(self):
        # These are the keys we'll access in __call__.
        # 'detection' is a virtual key that gets mapped to the real field
        # via field_mapping. 'filepath' and 'metadata' are standard fields.
        return ["filepath", "detection", "metadata"]
    
    def __call__(self, d):
        """
        Transform a FiftyOne sample dict into MedGemma fine-tuning format.
        
        Converts FiftyOne bbox format [x, y, w, h] in [0,1] to 
        MedGemma format [y0, x0, y1, x1] normalized to [0, 1000].
        """
        filepath = d["filepath"]
        detection = d["detection"]
        
        # Get the label from the detection
        label = detection.label
        
        # --- Bounding Box Conversion ---
        # FiftyOne stores bboxes as [x, y, width, height] with values in [0, 1]
        # MedGemma expects [y0, x0, y1, x1] normalized to [0, 1000]
        rx, ry, rw, rh = detection.bounding_box
        
        # Convert to [y0, x0, y1, x1] format, scaled to [0, 1000]
        x0 = int(rx * 1000)
        y0 = int(ry * 1000)
        x1 = int((rx + rw) * 1000)
        y1 = int((ry + rh) * 1000)
        
        # Format as [y0, x0, y1, x1] per the prompt instructions
        bbox_normalized = [y0, x0, y1, x1]
        
        # --- Construct Messages ---
        # Format the target response as JSON
        target_json = f'```json[{{"box_2d": {bbox_normalized}, "label": "{label}"}}]```'
        
        # Build the message payload in chat format
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": f"{LOCALIZATION_SYSTEM_PROMPT}\n\nLocate the {label} in this medical image."},
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": target_json},
                ],
            },
        ]
        
        return {
            "filepath": filepath,
            "image": Image.open(filepath).convert("RGB"),
            "messages": messages,
            "label": label,
        }

### Step 2: Create train/val split and flatten detections

Since our dataset doesn't have existing train/val tags, we'll create them.
Then we use `to_patches("detections")` to flatten the dataset so each detection 
becomes its own sample.

In [ ]:
import fiftyone.utils.random as four

# Filter to samples that have detections
has_detections_view = dataset.match(F("detections") != None)
print(f"Samples with detections: {len(has_detections_view)}")

# Create train/val split (80/20)
four.random_split(has_detections_view, {"train": 0.8, "val": 0.2})

In [ ]:
# Filter by split tags
train_view = has_detections_view.match_tags("train")
val_view = has_detections_view.match_tags("val")

print(f"Samples - train: {len(train_view)}, val: {len(val_view)}")

In [ ]:
# Flatten using to_patches() - each detection becomes its own sample
train_patches = train_view.to_patches("detections")
val_patches = val_view.to_patches("detections")

print(f"Patches - train: {len(train_patches)}, val: {len(val_patches)}")

### Step 3: Convert to PyTorch datasets

Use `to_torch()` with our `GetItem` class to create PyTorch-compatible datasets.
The `field_mapping` tells FiftyOne which actual field to use when we access "detection".

In [ ]:
# Set up field mapping - in patches view, each sample's detection data 
# lives in the original field "detections"
field_mapping = {"detection": "detections"}

# Create GetItem instances
train_getter = LocalizationGetItem(field_mapping=field_mapping)
val_getter = LocalizationGetItem(field_mapping=field_mapping)

# Convert to PyTorch datasets
train_dataset = train_patches.to_torch(train_getter)
val_dataset = val_patches.to_torch(val_getter)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")

In [ ]:
# Verify the data format
sample = train_dataset[0]
print("Sample keys:", sample.keys())
print("Label:", sample["label"])
print("Messages structure:")
for msg in sample["messages"]:
    print(f"  Role: {msg['role']}")

### Step 4: Load MedGemma with QLoRA configuration

We use 4-bit quantization (QLoRA) to reduce memory requirements while maintaining
fine-tuning capability. This allows fine-tuning on consumer GPUs.

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig

model_id = "google/medgemma-1.5-4b-it"

# Check if GPU supports bfloat16
if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] < 8:
    raise ValueError("GPU does not support bfloat16, please use a GPU that supports bfloat16.")

model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Configure 4-bit quantization for memory efficiency
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=model_kwargs["torch_dtype"],
    bnb_4bit_quant_storage=model_kwargs["torch_dtype"],
)

model = AutoModelForImageTextToText.from_pretrained(model_id, **model_kwargs)
processor = AutoProcessor.from_pretrained(model_id)

# Use right padding during training to avoid issues
processor.tokenizer.padding_side = "right"

### Step 5: Configure LoRA

LoRA (Low-Rank Adaptation) allows efficient fine-tuning by only training 
small adapter matrices instead of all model weights.

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=[
        "lm_head",
        "embed_tokens",
    ],
)

### Step 6: Define the collate function

The collate function processes batches by:
1. Applying the chat template to format messages
2. Processing images and text together
3. Creating labels with proper masking for padding and image tokens

In [ ]:
def collate_fn(examples: list[dict[str, Any]]):
    texts = []
    images = []
    
    for example in examples:
        # Convert image to RGB and wrap in list (processor expects list of images per sample)
        images.append([example["image"].convert("RGB")])
        
        # Apply chat template to format the conversation
        texts.append(processor.apply_chat_template(
            example["messages"], 
            add_generation_prompt=False, 
            tokenize=False
        ).strip())
    
    # Tokenize texts and process images
    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
    
    # Create labels from input_ids
    # We mask padding tokens and image tokens so they don't contribute to loss
    labels = batch["input_ids"].clone()
    
    # Get the image token ID to mask it
    image_token_id = processor.tokenizer.convert_tokens_to_ids(
        processor.tokenizer.special_tokens_map["boi_token"]
    )
    
    # Mask tokens that should not be used in loss computation
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    labels[labels == 262144] = -100  # Additional image token ID
    
    batch["labels"] = labels
    return batch

### Step 7: Configure training

We use TRL's `SFTConfig` and `SFTTrainer` for a clean training setup with
all the best practices built in.

In [ ]:
from trl import SFTConfig, SFTTrainer

num_train_epochs = 1  # Adjust based on your needs
learning_rate = 2e-4

training_args = SFTConfig(
    output_dir="medgemma-localization-lora",         # Directory to save the model
    num_train_epochs=num_train_epochs,               # Number of training epochs
    per_device_train_batch_size=4,                   # Batch size per device during training
    per_device_eval_batch_size=4,                    # Batch size per device during evaluation
    gradient_accumulation_steps=4,                   # Number of steps before performing a backward/update pass
    gradient_checkpointing=True,                     # Enable gradient checkpointing to reduce memory usage
    optim="adamw_torch_fused",                       # Use fused AdamW optimizer for better performance
    logging_steps=50,                                # Number of steps between logs
    save_strategy="epoch",                           # Save checkpoint every epoch
    eval_strategy="steps",                           # Evaluate every `eval_steps`
    eval_steps=50,                                   # Number of steps between evaluations
    learning_rate=learning_rate,                     # Learning rate
    bf16=True,                                       # Use bfloat16 precision
    max_grad_norm=0.3,                               # Max gradient norm
    warmup_ratio=0.03,                               # Warmup ratio
    lr_scheduler_type="linear",                      # Use linear learning rate scheduler
    push_to_hub=False,                               # Set to True to push model to Hub
    report_to="tensorboard",                         # Report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_kwargs={"skip_prepare_dataset": True},   # We preprocess manually
    remove_unused_columns=False,                     # Keep columns for data collator
    label_names=["labels"],                          # Input keys that correspond to labels
)

### Step 8: Create trainer and train!

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=peft_config,
    processing_class=processor,
    data_collator=collate_fn,
)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Save the fine-tuned model
trainer.save_model()

# Optional: Push to Hugging Face Hub
# trainer.push_to_hub()

### Clean up GPU memory

In [ ]:
del model
del trainer
import gc
gc.collect()
torch.cuda.empty_cache()

---
## Evaluating Your Fine-Tuned Model

Now that you've fine-tuned MedGemma for localization, go back through the earlier 
sections of this notebook to evaluate how well your model performs:

1. **Load your fine-tuned model** and run inference on the validation set
2. **Store predictions** in FiftyOne alongside the ground truth
3. **Use the evaluation techniques** from Sections 5 and 6:
   - Compute accuracy by modality, body part, and content type
   - Analyze errors using the App and similarity search
   - Tag patterns in failures

This iterative workflow—explore, model, evaluate, fine-tune—is how you systematically
improve your model's performance on specific failure modes.

---
## Bringing It All Together

Here's what you've learned to do:

| Step | What You Did | Why It Matters |
|------|-------------|----------------|
| Load & Explore | Understood data distribution before modeling | Caught potential issues early |
| Embeddings | Visualized MedSigLIP clusters | Diagnosed whether classes are separable |
| Inference | Ran MedGemma, stored predictions with data | Everything in one place for analysis |
| Evaluation | Sliced accuracy by modality, body part, etc. | Found *where* the model fails |
| Error Analysis | Visualized failures, tagged patterns | Understood *why* it fails |
| Fine-Tuning | Used GetItem + SFTTrainer for localization | Improved model on specific failure modes |

**The workflow you built here works for any dataset, any model, any challenge.**

Whether you're doing:
- Chest X-ray report generation
- Dermatology classification  
- CT severity assessment
- Histopathology analysis

The pattern is the same:
1. Organize your data in FiftyOne
2. Understand it before modeling
3. Run inference, store predictions
4. Slice, visualize, debug
5. Fine-tune and iterate

**Now go win that challenge.** 🏆

---
## Resources

- [FiftyOne Documentation](https://docs.voxel51.com/)
- [FiftyOne PyTorch Integration](https://docs.voxel51.com/api/fiftyone.utils.torch.html)
- [SLAKE Dataset on HuggingFace](https://huggingface.co/datasets/Voxel51/SLAKE)
- [MedGemma Model Card](https://huggingface.co/google/medgemma-1.5-4b-it)
- [MedSigLIP Model Card](https://huggingface.co/google/medsiglip-448)
- [TRL SFTTrainer Documentation](https://huggingface.co/docs/trl/sft_trainer)
- [PEFT LoRA Documentation](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora)
- [MedGemma Impact Challenge](https://www.kaggle.com/competitions/med-gemma-impact-challenge)